In [6]:
!pip install tensorflow 2.4.1 opencv-python matplotlib


ERROR: Could not find a version that satisfies the requirement 2.4.1 (from versions: none)
ERROR: No matching distribution found for 2.4.1


In [7]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [12]:
Model(inputs=[inputimage,verificationimage], outputs=[1,0])

NameError: name 'inputimage' is not defined

In [13]:
POS_PATH =os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', "negative")
ANC_PATH = os.path.join('data', 'anchor')


In [14]:
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)